In [1]:
# Load required modules
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.preprocessing
import pandas as pd, numpy as np
from datetime import datetime, date
from matplotlib import pyplot as plt
import seaborn as sns
import doubleml as dml
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [4]:
# Load example data set
url = 'https://raw.githubusercontent.com/DoubleML/doubleml-docs/main/doc/examples/data/orig_demand_data_example.csv'
data_example = pd.read_csv(url)
data_example

,Unnamed: 0,Date,StockCode,Country,Description,Quantity,revenue,UnitPrice
0,0,2010-12-01,10002,France,INFLATABLE POLITICAL GLOBE,48,40.80,0.850
1,1,2010-12-01,10002,United Kingdom,INFLATABLE POLITICAL GLOBE,12,10.20,0.850
2,2,2010-12-01,10125,United Kingdom,MINI FUNKY DESIGN TAPES,2,1.70,0.850
3,3,2010-12-01,10133,United Kingdom,COLOURING PENCILS BROWN TUBE,5,4.25,0.850
4,4,2010-12-01,10135,United Kingdom,COLOURING PENCILS BROWN TUBE,1,2.51,2.510
5,5,2010-12-01,11001,United Kingdom,ASSTD DESIGN RACING CAR PEN,3,10.08,3.360
6,6,2010-12-01,15044B,United Kingdom,BLUE PAPER PARASOL,1,2.95,2.950
7,7,2010-12-01,15056BL,United Kingdom,EDWARDIAN PARASOL BLACK,20,113.00,5.650
8,8,2010-12-01,15056N,United Kingdom,EDWARDIAN PARASOL NATURAL,50,236.30,4.726
9,9,2010-12-01,15056P,United Kingdom,EDWARDIAN PARASOL PINK,48,220.80,4.600


In [6]:
url2 = 'https://raw.githubusercontent.com/DoubleML/doubleml-docs/main/doc/examples/data/elasticity_subset.csv'
demand_data = pd.read_csv(url2, index_col = 0)
demand_data.head()

,Quantity,revenue,UnitPrice,month,DoM,DoW,stock_age_days,sku_avg_p,Australia,Austria,...,544,545,546,547,548,549,550,551,dLnP,dLnQ
189628,5,8.15,1.630000,9,6,1,278,0.85,False,False,...,0,0,0,0,0,0,0,0,0.408612,-1.013840
37914,19,41.93,2.206842,1,25,1,55,2.10,False,False,...,0,0,0,0,0,0,0,0,-0.048811,-0.306368
80103,24,20.40,0.850000,3,31,3,120,0.85,False,False,...,0,0,0,0,0,0,0,0,-0.127108,1.371479
75019,12,23.40,1.950000,3,24,3,113,2.08,False,False,...,0,0,0,0,0,0,0,0,-0.228256,-1.034483
99878,4,39.80,9.950000,5,5,3,155,9.95,False,False,...,0,0,0,0,0,0,0,0,-0.034922,-0.050881


In [7]:
demand_data.shape

(10000, 600)

In [12]:
feature_names = demand_data.columns.drop(['Quantity', 'revenue', 'UnitPrice', 'dLnQ', 'dLnP']).to_list()

In [13]:
data_dml = dml.DoubleMLData(
    demand_data,
    y_col = 'dLnQ', # outcome
    d_cols = 'dLnP', # treatment
    x_cols = feature_names, # confounders
)

In [14]:
print(data_dml)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: dLnQ
Treatment variable(s): ['dLnP']
Covariates: ['month', 'DoM', 'DoW', 'stock_age_days', 'sku_avg_p', 'Australia', 'Austria', 'Bahrain', 'Belgium', 'Brazil', 'Canada', 'Channel Islands', 'Cyprus', 'Czech Republic', 'Denmark', 'EIRE', 'European Community', 'Finland', 'France', 'Germany', 'Greece', 'Hong Kong', 'Iceland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'RSA', 'Saudi Arabia', 'Singapore', 'Spain', 'Sweden', 'Switzerland', 'USA', 'United Arab Emirates', 'United Kingdom', 'Unspecified', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51',

In [ ]:
ml_l_lin_reg = 